# Named entities


In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger
from flair.tokenization import SegtokSentenceSplitter

sentence = Sentence("I love Berlin .")

tagger = SequenceTagger.load('ner')

tagger.predict(sentence)

In [ ]:
print(sentence)
print('The following NER tags are found:')

# iterate over entities and print
for entity in sentence.get_spans('ner'):
    print(entity)

In [ ]:
path = "../data/Original/iued_test_original.txt"

with open(path, 'r') as file:
    data = file.read().replace("\n", "")


Apparently Flair can only work on individual sentences, it does however provide a function to convert text into a long "sentence".

In [ ]:
splitter = SegtokSentenceSplitter()

sentences = splitter.split(data)

for sentence in sentences:
    print(sentence)

In [ ]:
tagger1 = SequenceTagger.load('ner-fast')

tagger.predict(sentences)

Flair seems to be quite slow compared to spacy and stanza

In [ ]:
for sentence in sentences:
    print(sentence.to_tagged_string())

In [ ]:
for sentence in sentences:
    for entity in sentence.get_spans():
        print(entity.text)
        pos_str = entity.position_string.split("-")
        pos_int = [int(pos_str[0])+1, int(pos_str[-1])+2]
        print(pos_int)
        print(entity.labels)

In [ ]:
print(sentences[0][0])

Token Indices start at 1 in flair